In [57]:
import pandas as pd

In [58]:
df = pd.read_csv('rentprediction_dataset_v5.csv')
pd.set_option('display.max_columns', None)

In [59]:
from sklearn.model_selection import train_test_split

y = df['rent']
X = df.drop(columns={'rent'})

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.177, random_state=42)

In [52]:
import pandas as pd
import numpy as np

# Define the transformation functions
def log_transform(x):
    return np.log(x + 1)  # Add 1 to avoid log(0)

def sqrt_transform(x):
    return np.sqrt(x)

def inverse_transform(x):
    return 1 / (x + 1)  # Avoid division by zero

def square_transform(x):
    return np.square(x)

def exp_transform(x):
    return np.exp(x)

# List of transformations
transformations = {
    'none': lambda x: x,
    'log': log_transform,
    'sqrt': sqrt_transform,
    'inverse': inverse_transform,
    'square': square_transform,
    'exp': exp_transform
}

# Ensure X_train and y_train are properly formatted
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_train = np.log(y_train)

# If y_train is a DataFrame with a single column, convert it to a Series
if isinstance(y_train, pd.DataFrame):
    y_train = y_train.iloc[:, 0]

# Function to determine if a variable is binary
def is_binary(series):
    return series.nunique() == 2

# Dictionary to store the best transformation for each column
best_transformations = {}

# Iterate over each column in X_train
for column in X_train.columns:
    if is_binary(X_train[column]):
        # Skip transformation for binary variables
        best_transformations[column] = 'none'
        continue

    column_correlations = {}
    for name, func in transformations.items():
        transformed_column = func(X_train[column])
        correlation = y_train.corr(transformed_column)
        column_correlations[name] = correlation

    # Find the transformation with the highest absolute correlation for this column
    best_transformation = max(column_correlations, key=lambda k: abs(column_correlations[k]))
    best_transformations[column] = best_transformation

# Create a new DataFrame with the best transformations applied and updated column names
transformed_df = pd.DataFrame()

for column in X_train.columns:
    best_transformation = best_transformations[column]
    if best_transformation == 'none':
        transformed_df[column] = X_train[column]
    else:
        transformed_df[f'{column}_{best_transformation}'] = transformations[best_transformation](X_train[column])

# Print the best transformation for each column
print("Best transformations for each column:")
for column, transformation in best_transformations.items():
    print(f"{column}: {transformation}")

# Show the transformed DataFrame
print("\nTransformed DataFrame:")
print(transformed_df.head())

# Optionally: if you want to remove the original columns and only keep the transformed ones
# Find the columns that were transformed (those that don't have 'none' as the best transformation)
transformed_columns = [f'{column}_{best_transformation}' if best_transformation != 'none' else column for column, best_transformation in best_transformations.items()]

# Update X_train to keep only the transformed columns
X_train_transformed = transformed_df[transformed_columns]

# Show the final DataFrame
print("\nFinal DataFrame with best transformations applied:")
print(X_train_transformed.head())


/home/anass/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/anass/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/anass/.local/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/anass/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2742: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
/tmp/ipykernel_641/3841473505.py:71: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  transformed_df[f'{column}_{best_transforma

Best transformations for each column:
age: log
sqmtr: sqrt
newbuild: none
rooms: none
elabel: log
parking: none
bedrooms: none
bathrooms: square
floor: log
garden: none
balcony: none
rooftop: none
apartment: none
duplexmulti: none
luxury: none
singlefam: none
houseboat: none
pc4: square
percnative_pc4: inverse
sizehoudehold_pc4: inverse
percpropertyown_pc4: sqrt
taxablepropertyvaluation: log
populationdensity_pc4: sqrt
latitude: exp
longitude: inverse
eucl_dist_to_Amstelland: inverse
eucl_dist_to_Amstelpark: inverse
eucl_dist_to_Amsterdamse Bos: sqrt
eucl_dist_to_Beatrixpark: sqrt
eucl_dist_to_Brasapark: square
eucl_dist_to_De Bretten: sqrt
eucl_dist_to_Diemerpark: square
eucl_dist_to_Erasmuspark: square
eucl_dist_to_Flevopark: square
eucl_dist_to_Gaasperplas: square
eucl_dist_to_Gijsbrecht van Aemstelpark: sqrt
eucl_dist_to_Houthavenpark: inverse
eucl_dist_to_Martin Luther Kingpark: exp
eucl_dist_to_Nelson Mandelapark: square
eucl_dist_to_Noorder IJplas: sqrt
eucl_dist_to_Noorderpark:

In [56]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Define the transformation functions
def log_transform(x):
    return np.log(x + 1)  # Add 1 to avoid log(0)

def sqrt_transform(x):
    return np.sqrt(x)

def inverse_transform(x):
    return 1 / (x + 1)  # Avoid division by zero

def square_transform(x):
    return np.square(x)

def exp_transform(x):
    return np.exp(x)

# List of transformations
transformations = {
    'none': lambda x: x,
    'log': log_transform,
    'sqrt': sqrt_transform,
    'inverse': inverse_transform,
    'square': square_transform,
    'exp': exp_transform
}

# Ensure X_train and y_train are properly formatted
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

# If y_train is a DataFrame with a single column, convert it to a Series
if isinstance(y_train, pd.DataFrame):
    y_train = y_train.iloc[:, 0]

# Function to determine if a variable is binary
def is_binary(series):
    return series.nunique() == 2

# Initialize variables to store the best overall performance
best_r2 = -np.inf
best_mse = np.inf
best_combination = None
best_transformed_df = None

# Iterate over each transformation of y_train
for y_name, y_func in transformations.items():
    transformed_y_train = y_func(y_train)
    
    # Dictionary to store the best transformation for each column
    best_transformations = {}
    transformed_X_train = pd.DataFrame()

    # Iterate over each column in X_train
    for column in X_train.columns:
        if is_binary(X_train[column]):
            # Skip transformation for binary variables
            best_transformations[column] = 'none'
            transformed_X_train[column] = X_train[column]
            continue

        best_correlation = -np.inf
        best_transformation = 'none'
        for x_name, x_func in transformations.items():
            transformed_column = x_func(X_train[column])
            correlation = transformed_y_train.corr(transformed_column)
            if abs(correlation) > abs(best_correlation):
                best_correlation = correlation
                best_transformation = x_name

        # Apply the best transformation to the column
        transformed_X_train[f'{column}_{best_transformation}'] = transformations[best_transformation](X_train[column])
        best_transformations[column] = best_transformation

    # Perform linear regression
    model = LinearRegression()
    model.fit(transformed_X_train, transformed_y_train)
    predictions = model.predict(transformed_X_train)
    
    # Calculate performance metrics
    r2 = r2_score(transformed_y_train, predictions)
    mse = mean_squared_error(transformed_y_train, predictions)

    # Check if this is the best model so far
    if r2 > best_r2 or (r2 == best_r2 and mse < best_mse):
        best_r2 = r2
        best_mse = mse
        best_combination = (y_name, best_transformations)
        best_transformed_df = transformed_X_train.copy()

# Print the best combination of transformations
print("Best overall combination of transformations:")
print(f"y_train: {best_combination[0]}")
for column, transformation in best_combination[1].items():
    print(f"{column}: {transformation}")

# Show the final transformed DataFrame
print("\nFinal transformed DataFrame with best transformations applied:")
print(best_transformed_df.head())


/home/anass/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/anass/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/anass/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/anass/.local/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/anass/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2742: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
/home/anass/.local/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/anass/.local/lib/pyth

ValueError: Input y contains infinity or a value too large for dtype('float64').